In [9]:
from torchtext import vocab, data    
PATH='./'  
TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

In [10]:
%ls {PATH}trn

nietzsche.txt        nietzsche_train.txt


In [11]:
TEXT = data.Field(lower=True, tokenize=list)
bs = 64
bptt = 8
n_fac = 42
n_hidden = 256

In [18]:
from fastai.text import *

In [21]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = TextDataBunch.from_folder('.', train='trn', valid='val')

In [36]:
TEXT = TEXT.build_vocab(md)

TypeError: 'TextClasDataBunch' object is not iterable

In [77]:
import torch
from torch import nn

In [78]:
class CharSeqStatefulRNN(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
            
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp, h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=1).view(-1, self.vocab_size)

                                                           
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))
            